In [1]:
import os
import sys

dir = os.path.abspath('')
parent = os.path.dirname(dir)
if not parent in sys.path:
    sys.path.append(parent)


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.ops import box_iou
from torchvision.transforms import Compose, Resize, ToTensor

from data.custom_celeba import CelebA, xywh2xyxy
from models.my_net import MyNet


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device {device}')


Using device cuda


In [3]:
transform = Compose([
    ToTensor(),
    Resize((256, 256))
])

target_transform = Compose([
    xywh2xyxy,
    torch.Tensor.float
])

training_data = CelebA(
    root='CelebA',
    split='train',
    transform=transform,
    target_transform=target_transform,
    resize=True
)

test_data = CelebA(
    root='CelebA',
    split='test',
    transform=transform,
    target_transform=target_transform,
    resize=True
)


In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=6)
test_dataloader = DataLoader(test_data, batch_size=batch_size, num_workers=6)


In [5]:
model = MyNet().to(device)


In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * dataloader.batch_size
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (box_iou(pred, y).diagonal() > 0.5).sum().item()
    test_loss /= num_batches
    correct /= size

    print(
        f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')
    
    return correct


In [7]:
epochs = 30
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

correct = 0
for epoch in range(1, epochs+1):
    print(f'Epoch {epoch}\n-------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    new_correct = test(test_dataloader, model, loss_fn)

    if new_correct > correct:
        torch.save(model.state_dict(), 'model.pth')
        print('Saved PyTorch Model State to model.pth')
        correct = new_correct
    else:
        print('Accuracy decreased rolling back')
        model.load_state_dict(torch.load('model.pth'))

print('Done!')


Epoch 1
-------------------------------
loss: 16346.705078  [    0/162770]
loss: 15673.807617  [ 6400/162770]
loss: 10718.539062  [12800/162770]
loss: 4259.604492  [19200/162770]
loss: 1204.190918  [25600/162770]
loss: 335.938843  [32000/162770]
loss: 261.414795  [38400/162770]
loss: 177.720612  [44800/162770]
loss: 174.067719  [51200/162770]
loss: 178.799591  [57600/162770]
loss: 176.509766  [64000/162770]
loss: 180.869659  [70400/162770]
loss: 166.697342  [76800/162770]
loss: 159.362335  [83200/162770]
loss: 118.141708  [89600/162770]
loss: 121.368576  [96000/162770]
loss: 100.765869  [102400/162770]
loss: 110.407532  [108800/162770]
loss: 88.792816  [115200/162770]
loss: 118.641167  [121600/162770]
loss: 75.856644  [128000/162770]
loss: 61.609322  [134400/162770]
loss: 80.497894  [140800/162770]
loss: 90.502686  [147200/162770]
loss: 55.683220  [153600/162770]
loss: 73.807587  [160000/162770]
Test Error: 
 Accuracy: 96.3%, Avg loss: 96.491079 

Saved PyTorch Model State to model.pth

KeyboardInterrupt: 